# Vertices for Hubbard atom

In [ ]:
using Revise
using PyPlot

using OvercompleteIR
using OvercompleteIR.Atom
using OvercompleteIR.SparseIR
using OvercompleteIR.LinearAlgebra

using ITensors
using Quantics

d = DensityChannel()
m = MagneticChannel()
s = SingletChannel()
t = TripletChannel()

In [ ]:
U = 0.1
β = 0.2
at = Atom.HubbardAtom(U, β)
conv = Atom.PHConventionThunstroem()

In [ ]:
N = 4
half_N = N ÷ 2
nw = 2^N
half_nw = 2^(N-1)

ννω = OvercompleteIR.conv_type(conv).(
    Iterators.product(
        FermionicFreq(-2*half_nw+1):FermionicFreq(2*half_nw-1),
        FermionicFreq(-2*half_nw+1):FermionicFreq(2*half_nw-1),
        BosonicFreq(-2*half_nw):BosonicFreq(2*half_nw-2)
    )
)

νω =  Iterators.product(
        FermionicFreq(-2*half_nw+1):FermionicFreq(2*half_nw-1),
        BosonicFreq(-2*half_nw):BosonicFreq(2*half_nw-2)
    )
;

In [ ]:
channel = s
Γ  = Atom.Γ.(channel, at, ννω)
X₀ = Atom.χ₀.(channel, at, νω)
F  = Atom.F.(channel, at, ννω)
Φ = F .- Γ

X₀_full = zero(Γ)
for i in 1:nw
    X₀_full[i, i, :] = X₀[i, :]
end
;

In [ ]:
function evalF(channel, ννω, Γ, X₀, F)
    Φ = Array{ComplexF64}(undef, size(ννω))
    for I in CartesianIndices(Φ)
        (ν, ν´, ω) = Tuple(I)
        Φ[I] = sum(Γ[ν,ν₁,ω] * X₀[ν₁,ω] * F[ν₁,ν´,ω] for ν₁ in axes(X₀, 1))
    end
    κ = channel isa SingletChannel ? 1 : -1
    Φ .*= κ/β
    return Γ .+ Φ
end

F_reconst = evalF(channel, ννω, Γ, X₀, F)
;

In [ ]:
@show maximum(abs, F)
@show maximum(abs, F_reconst .- F)

In [ ]:
sitesν = [Index(2, "Qubit,ν=$n") for n in 1:N]
sitesν′ = [Index(2, "Qubit,ν′=$n") for n in 1:N]
sitesω = [Index(2, "Qubit,ω=$n") for n in 1:N]
sitesνν′ω = collect(Iterators.flatten(zip(sitesν, sitesν′, sitesω)))
sitesνω = collect(Iterators.flatten(zip(sitesν, sitesω)))

In [ ]:
revcat(x...) = vcat((reverse(x_) for x_ in x)...)
Γ_qtt = MPS(ITensor(Γ, revcat(sitesν, sitesν′, sitesω)), reverse(sitesνν′ω); cutoff=1e-10)
F_qtt = MPS(ITensor(F, revcat(sitesν, sitesν′, sitesω)), reverse(sitesνν′ω); cutoff=1e-10)
X₀_qtt = MPS(ITensor(X₀, revcat(sitesν, sitesω)), reverse(sitesνω); cutoff=1e-10)

In [ ]:
function _mul(A, B)
    sitesshared = [Index(2, "νs=$n") for n in 1:N]
    
    A_ = deepcopy(A)
    Quantics.replace_siteinds_part!(A_, sitesν′, sitesshared)
    
    B_ = deepcopy(B)
    Quantics.replace_siteinds_part!(B_, sitesν, sitesshared)
    
    C = Quantics.automul(A_, B_; tag_row="ν", tag_shared="νs", tag_col="ν′")
    Quantics.cleanup_linkinds!(C)
    
    return C
end

_reconst(M) = reshape(Array(reduce(*, M), revcat(sitesν, sitesν′, sitesω)), 2^N, 2^N, 2^N)

In [ ]:
X₀_full_qtt = Quantics.asdiagonal(X₀_qtt, sitesν′; tag="ν")
Φ_qtt = _mul(Γ_qtt, _mul(X₀_full_qtt, F_qtt))
κ = channel isa SingletChannel ? 1 : -1
Φ_qtt *= κ/β

In [ ]:
Φ_reconst = _reconst(Φ_qtt)

@show maximum(abs, Φ_reconst)
@show maximum(abs, Φ)

findmax(abs, Φ)

In [ ]:
plot(real.(Φ[:, 8, 9]))
plot(real.(Φ_reconst[:, 8, 9]))

In [ ]:
@show maximum(abs, Φ)
@show maximum(abs, Φ_reconst)
plt[:pcolormesh](real.(Φ[:, :, 9]))
colorbar()

In [ ]:
plt[:pcolormesh](real.(Φ_reconst[:, :, 9]))
colorbar()